### Adaptamos el dataset a una cantidad decente para trabajar

In [1]:
import pandas as pd

#animes=pd.read_csv('/content/drive/MyDrive/Data Science 2023/Sprints/Sprint 3/anime.csv')
#animesynopsis= pd.read_csv('/content/drive/MyDrive/Data Science 2023/Sprints/Sprint 3/anime_with_synopsis.csv')
animelist= pd.read_csv(r'C:\Users\fran_\Downloads\animelist.csv')
#ratings= pd.read_csv('/content/drive/MyDrive/Data Science 2023/Sprints/Sprint 3/rating_complete.csv')
#watchingstatus= pd.read_csv('/content/drive/MyDrive/Data Science 2023/Sprints/Sprint 3/watching_status.csv')


In [6]:
# Agrupar por "anime_id" y contar las calificaciones para cada anime
animes_p = animelist.groupby(["anime_id"], as_index=False)["rating"].count()

# Renombrar la columna resultante como "Ratings Count"
animes_p = animes_p.rename(columns={"rating": "Ratings Count"})

# Ordenar de mayor a menor según la cantidad de calificaciones
animes_p = animes_p.sort_values(by="Ratings Count", ascending=False)

In [29]:
animes_p=animes_p.iloc[0:10000].anime_id

AttributeError: 'list' object has no attribute 'iloc'

In [20]:
animes_p
animes_p.to_csv(r"C:\Users\fran_\Downloads\10000populares.csv")

In [24]:
animes_p=list(animes_p)

interactions_selected = animelist[['user_id', 'anime_id', 'rating','watching_status']]

interactions_selected = interactions_selected[interactions_selected['anime_id'].isin(animes_p)]

In [28]:
print(interactions_selected)
print(animelist)
interactions_selected.anime_id.nunique()
#Ya tenemos interactions_ selected con los animes mas votados ahora tenemos que filtrar por solo 200000 usuarios al azar o podemos seleccionar aquellos que mas peliculas vieron tmb
#Serian los usuarios mas activos (?

           user_id  anime_id  rating  watching_status
0                0        67       9                1
1                0      6702       7                1
2                0       242      10                1
3                0      4898       0                1
4                0        21      10                1
...            ...       ...     ...              ...
109224742   353404       507       7                2
109224743   353404       392       9                2
109224744   353404       882       6                2
109224745   353404       883       8                2
109224746   353404       149       0                6

[108179766 rows x 4 columns]
           user_id  anime_id  rating  watching_status  watched_episodes
0                0        67       9                1                 1
1                0      6702       7                1                 4
2                0       242      10                1                 4
3                0      4898      

10000

In [33]:
# Agrupar por "anime_id" y contar las calificaciones para cada anime
users_a = animelist.groupby(["user_id"], as_index=False)["rating"].count()

# Renombrar la columna resultante como "Ratings Count"
users_a = users_a.rename(columns={"rating": "Ratings Count"})

# Ordenar de mayor a menor según la cantidad de calificaciones
users_a = users_a.sort_values(by="Ratings Count", ascending=False)

In [42]:
users_a=users_a['user_id'].iloc[0:25000]  #El que menos voto, voto 836 ratings y nos llevamos 250000 usuarios 

KeyError: 'user_id'

In [45]:
users_a.to_csv(r"C:\Users\fran_\Downloads\25kusers.csv")
users_a=list(users_a)
interactions_selected = interactions_selected[interactions_selected['user_id'].isin(users_a)]

In [50]:
print(interactions_selected)
print(animelist)
print(interactions_selected.anime_id.nunique())
print(interactions_selected.user_id.nunique())

           user_id  anime_id  rating  watching_status
3988            17     34572       0                1
3989            17      2994       0                1
3990            17     40852       0                1
3991            17     41694       0                1
3992            17     40530       0                1
...            ...       ...     ...              ...
109224268   353398     34086       0                6
109224269   353398     17909       0                6
109224270   353398     32924       0                6
109224271   353398     24627       0                6
109224272   353398     34467       0                6

[32874567 rows x 4 columns]
           user_id  anime_id  rating  watching_status  watched_episodes
0                0        67       9                1                 1
1                0      6702       7                1                 4
2                0       242      10                1                 4
3                0      4898       

In [52]:
from sklearn.model_selection import train_test_split

interactions_selected.drop(columns= 'watching_status')

,user_id,anime_id,rating
3988,17,34572,0
3989,17,2994,0
3990,17,40852,0
3991,17,41694,0
3992,17,40530,0
...,...,...,...
109224268,353398,34086,0
109224269,353398,17909,0
109224270,353398,32924,0
109224271,353398,24627,0


In [53]:
train, test = train_test_split(interactions_selected, test_size=0.2, random_state=42)
train=train.drop(columns= 'watching_status')
print(train.user_id.nunique())
test.user_id.nunique()   #Ejecutamos esta linea o la de arriba, si no se pisan 

25000


25000

In [54]:
test[~test.user_id.isin(train.user_id.unique())].user_id.nunique()      #isin se fija si un valor esta en el dataframe, si esta, lo marca como true, si no False, como busca los que no ~
#Practicamente todos, pero con el dataset completo, tenemos mejores resultados jeje
#Nos dice todos los usuarios que no estan en el set de train 

0

In [55]:
#Tiramos los duplicados
test= test.drop_duplicates()
train=train.drop_duplicates()
train

,user_id,anime_id,rating
82776471,268041,32182,9
51486712,166917,2798,6
35249659,114397,34350,4
4824652,15946,877,0
7758832,25411,31139,6
...,...,...,...
77744287,251721,24439,7
53378052,173110,32553,8
77380738,250647,20939,8
106646913,344943,17437,0


In [56]:
# Pivoteamos la tabla, cambiamos columnas por indices y damos vuelta
interactions_matrix = train.pivot(index="user_id", columns="anime_id", values="rating")

interactions_matrix.head() #Tememos que llenar los NaN con nulos que son las peliculas que no vieron
interactions_matrix.shape

(25000, 10000)

In [57]:
interactions_matrix = interactions_matrix.fillna(0)

interactions_matrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,47639,47778,47790,48316,48375,48405,48413,48417,48418,48438
user_id,,,,,,,,,,,,,,,,,,,,,
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,8.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
353365,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
353390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
interactions_matrix.to_csv(r"C:\Users\fran_\Downloads\interactions_matrix.csv")
train.to_csv(r"C:\Users\fran_\Downloads\train.csv")
test.to_csv(r"C:\Users\fran_\Downloads\test.csv")